In [2]:
import sys
sys.path.append('../')

In [3]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd
import requests

In [4]:
# import sklearn

In [5]:
table_name='epa'

In [6]:
##lUIS SILVA CÓDIGO
r = requests.get('https://cityscope-api.smartaraucania.org/api/table/epa/GEOGRID')


### Load the Saved Data

In [7]:
geogrid=gpd.read_file('../tables/{}/geogrid.geojson'.format(table_name))
#zones=gpd.read_file(r.text, driver ='GeoJSON')
zones=gpd.read_file('../tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype('int64')
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('../tables/{}/simpop_df.csv'.format(table_name))

### LODES industry is not useful- look up NAICS from wac instead
- TODO: do this in the script which prepares the USA data

In [8]:
import random
naics_cols=[col for col in zones.columns if 'naics' in col]
simpop_df['naics']=0
for ind, row in simpop_df.iterrows():
    if zones.loc[row['work_geoid'], 'emp_total']>0:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, weights=zones.loc[row['work_geoid'], naics_cols].values, k=1)[0]
    else:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, k=1)[0]

In [9]:
simpop_df=simpop_df.drop('industry', axis=1)

### Create the mobility system

In [10]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
getting external roads


c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\osmnx\simplification.py:499: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subcluster_centroid = node_points.loc[wcc].unary_union.centroid
c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\osmnx\simplification.py:500: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "x"] = subcluster_centroid.x
c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\osmnx\simplification.py:501: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "y"] = subcluster_centroid.y
c:\Users\user1\anaconda3\envs\spacialModules\lib\site-packages\osmnx\simplification.py:503: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "cluster"] 

	 Simplified from 1404 to 967 edges and 950 to 604 nodes
	 Pre-computing paths


In [11]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [12]:
H=Handler(table_name=table_name)
#H.reset_geogrid_data()

https://cityscope-api.smartaraucania.org/api/tables/list/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epa/meta/hashes/ Attempt: 0


In [ ]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
                       mode_choice_model=Default_Mode_Choice_model(),
                       mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)

H.add_indicators([
    d, 
    p, 
    m
])

# H.add_indicator(m)

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
Init simulation111
Se están creando las zonas


c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\CS_Indicators.py:626: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid


POINT (-122.1559159590466 37.46990865644195)
POINT (-122.15537676408951 37.470155729007814)
POINT (-122.15483756561424 37.470402799059364)
POINT (-122.15429836362081 37.47064986659653)
POINT (-122.15375915810915 37.47089693161926)
POINT (-122.15321994907931 37.47114399412746)
POINT (-122.15268073653134 37.47139105412107)
POINT (-122.15214152046507 37.47163811160004)
POINT (-122.15160230088073 37.47188516656432)
POINT (-122.15106307777812 37.472132219013815)
POINT (-122.15052385115736 37.472379268948444)
POINT (-122.14998462101846 37.47262631636819)
POINT (-122.14944538736133 37.472873361272974)
POINT (-122.14890615018604 37.47312040366271)
POINT (-122.14836690949255 37.47336744353734)
POINT (-122.14782766528091 37.47361448089681)
POINT (-122.14728841755107 37.47386151574106)
POINT (-122.1467491663031 37.474108548070014)
POINT (-122.14620991153693 37.4743555778836)
POINT (-122.14567065325262 37.474602605181744)
POINT (-122.14513139145011 37.47484962996442)
POINT (-122.14459212612942 37.

c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\CS_Indicators.py:593: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_zones=model_zones.append(grid_zones)


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRID/ Attempt: 0


...

Dens Ind: 0:00:00.011000
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Prox Ind: 0:00:00.363121
Prox HM: 0:00:00.169712
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Starting MM Update


c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\CS_Indicators.py:756: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)


Schedules and locations
OD
Trip table
Route table


c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:409: FutureWarning: The frame.append met

DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:03.198574
	 trips post took 0:00:00.623931


In [ ]:
# self=m
# import datetime
# print('Starting MM Update')
# start_calc=datetime.datetime.now()
# new_simpop=self.geogrid_updates(geogrid_data)
# new_simpop_df=pd.DataFrame(new_simpop)
# combined_simpop=self.base_simpop_df.append(new_simpop_df)
# sample_simpop_df=combined_simpop.sample(min(self.N_max, len(combined_simpop)))
# od_output, deckgl_trips, ind=self.simulate(sample_simpop_df)

In [ ]:
geogrid_data=H.get_geogrid_data()
#print(m.return_indicator(geogrid_data))
#print(geogrid_data)
#print(p.return_indicator(geogrid_data))
#print(d.return_indicator(geogrid_data))

https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0


In [ ]:
H.listen()

Table URL: https://cityscope.smartaraucania.org/CS_cityscopeJS/?cityscope=epaversion
Testing indicators
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.009735
Prox Ind: 0:00:00.347098
Prox HM: 0:00:00.060358
Starting MM Update


c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\CS_Indicators.py:756: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

Schedules and locations
OD
Trip table
Route table
DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:02.295359
	 trips post took 0:00:01.857258
Performing initial update
Updating geogrid_data
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
Updating table with hash: HJWKpey2P71uHmgwd6WGa2twf9cGuaVCaukePckwhMBe
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.004257
Prox Ind: 0:00:00.329876
Prox HM: 0:00:00.048005
Starting MM Update


c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\CS_Indicators.py:756: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\user1\Desktop\Spacial Modules\SpacialModulesPrueba\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

Schedules and locations
OD
Trip table
Route table
DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:02.148399
	 trips post took 0:00:00.472861
Done with update
Local grid hash: HJWKpey2P71uHmgwd6WGa2twf9cGuaVCaukePckwhMBe
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
Going into rest mode
Time active: 472.70985651016235
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/has

KeyboardInterrupt: 